In [163]:
import jax.numpy as jnp
import jax.random as jrandom

import matplotlib.pyplot as plt
import pandas as pd
from jax import jit, grad, jacfwd, jacrev

import pycutest

import scipy 

from scipy.stats import linregress

from tqdm import tqdm
import pickle

import numpy as np

from tqdm import tqdm

from jax.config import config
config.update("jax_enable_x64", True)

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [12,12]
plt.style.use('ggplot')
plt.rcParams['lines.markersize'] = 8
plt.rcParams.update({'font.size': 12})

%load_ext autoreload
%autoreload 2

import time
import pickle


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [164]:
import sys
sys.path.append("..")
from Functions import PyCutestWrapper
sys.path.append("../OptimalSetComps")
from generate_sing_vals_V import get_lambda_star, loss_getter, get_alpha, get_V_star, get_lambda_tilde, lmbda_loss


In [167]:
with open("results.pkl", "rb") as f:
    opt_set_data = pickle.load(f)




EOFError: Ran out of input